In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

import pandas as pd

import os
import re

from dotenv import load_dotenv
load_dotenv()

def get_file_names(folder_path, format=".pdf"):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.
    """
    import os
    
    try:
        all_files = os.listdir(folder_path)
        pdf_files = [file.replace(format,"") for file in all_files if file.lower().endswith(format)]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []
    

In [3]:
from loader import load_process_split_doc, load_process_split_doc_law

# tax_etc 파일들 정의의
doc_options = [
    {
        "filename":"연말정산_신고안내", 
        "page_ranges": [(17, 426)],
        "table_pages_range": "17-425",
        "replace_string": [
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"},
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"  },
            {"target_str": r"II. 2024년 귀속 연말정산 주요 일정\n\d+|"},
            {"target_str": r"III\. 원천징수의무자의 연말정산 중점 확인사항\n\d+|"},
            {"target_str": r"원천징수의무자를 위한 \n2024년 연말정산 신고안내\n\d+|"},
            {"target_str": r"Ⅰ\. 근로소득\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수 및 연말정산\n\d+|"},
            {"target_str": r"III\. 근로소득공제, 인적공제, 연금보험료공제\n\d+|"},
            {"target_str": r"IV\. 특별소득공제(소법 §52)\n\d+|"},
            {"target_str": r"V\. 그 밖의 소득공제(조특법)\n\d+|"},
            {"target_str": r"VI\. 세액감면(공제) 및 농어촌특별세\n\d+|"},
            {"target_str": r"I\. 2024년 귀속 연말정산 종합사례\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수영수증(지급명세서) 작성요령\n\d+|"},
            {"target_str": r"IV\. 수정 원천징수이행상황신고서 작성사례(과다공제)\n\d+|"},
            {"target_str": r"VI\. 홈택스를 이용한 연말정산 신고(근로소득 지급명세서 제출)\n\d+|"},
            {"target_str": r"I\. 사업소득 연말정산\n\d+|"},
            {"target_str": r"II\. 연금소득 연말정산\n\d+|"},
            {"target_str": r"I\. 종교인소득이란?\n\d+|"},
            {"target_str": r"IV\. 종교인소득(기타소득)에 대한 연말정산\n\d+|"},
            {"target_str": r"부록1\. 연말정산 관련 서비스\n\d+|"},
            {"target_str": r"부록2\. 연말정산간소화 서비스\n\d+|"},
            {"target_str": r"부록5\. 연말정산 주요 용어 설명\n\d+|"},
            {"target_str": r"부록6\. 소득·세액공제신고서 첨부서류\n\d+|"},
            {"target_str": r"\bNaN\b"},
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])","replace_str": r"\1\2"},
            {"target_str": r"\s+", "replace_str": r" "},
        ]
    },
    {
        "filename":"연말정산_주택자금·월세액_공제의이해", 
        "page_ranges": [(9, 12), (15, 24), (27, 28), (31, 39), (43, 70)],
        "table_pages_range": "9-12,15-24,27-28,31-39,43-70",
        "replace_string": [
            {"target_str": (r"연말정산 주택자금･월세액 공제의 이해\n\d+|" r"\bNaN\b"),  },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"2024_핵심_개정세법", 
        "page_ranges": [(19, 44), (47, 71), (75, 161)],
        "table_pages_range": "19-44,47-71,75-161",
        "replace_string": [
            {"target_str": (r"2\n0\n2\n5\n\s*달\n라\n지\n는\n\s*세\n금\n제\n도|"  
                r"\n2\n0\n2\n4\n\s*세\n목\n별\n\s*핵\n심\n\s*개\n정\n세\n법|"
                r"\n2\n0\n2\n4\n\s*개\n정\n세\n법\n\s*종\n전\n-\n개\n정\n사\n항\n\s*비\n교\n|"
                r"\s*3\s*❚국민･기업\s*납세자용\s*|"
                r"\s*2\s*0\s*2\s*4\s|"
                r"\s한국세무사회\s|" 
                r"\n7\n❚국민･기업 납세자용|"
                r"\n71\n❚상세본|"),  
            },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"주요_공제_항목별_계산사례", 
        "page_ranges": [(1,1), (4, 21)],
        "table_pages_range": "1,4-21",
        "replace_string": [
            {"target_str": r"\bNaN\b"},
            {"target_str": r"\s+", "replace_str": " " },
        ]
    }
    
]

# vector store 에 넣을 데이터 document 모으기 -> all_documents
all_documents = []
# load - 세법 
law_files = get_file_names("data/tax_law")
for file in law_files:
    all_documents.extend(load_process_split_doc_law(file))
# load - 참고자료
for doc in doc_options:
    all_documents.extend(load_process_split_doc(**doc))
    

c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 15 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 37 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 223 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 1 of document data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\par

In [3]:
# len(all_documents)

In [4]:
# 📌 vector store 생성
COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

def set_vector_store(documents):
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

    return Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )
    
vector_store = set_vector_store(all_documents)

In [5]:
COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

# 📌 vector store 불러오기

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = Chroma(
        embedding_function=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
)

In [6]:
# vector_store._collection.count()

In [7]:
# Retreiver 생성
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 5}
)

In [8]:
# Prompt Template 생성
messages = [
    ("ai", """
    당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 사용자의 질문에 대해 저장된 세법 조항 데이터와 관련 정보를 기반으로 정확하고 신뢰성 있는 답변을 제공하세요. 

    **역할 및 기본 규칙**:
    - 당신의 주요 역할은 세법 정보를 사용자 친화적으로 전달하는 것입니다.
    - 데이터에 기반한 정보를 제공하며, 데이터에 없는 내용은 임의로 추측하지 않습니다.
    - 불확실한 경우, "잘 모르겠습니다."라고 명확히 답변하고, 사용자가 질문을 더 구체화하도록 유도합니다.

    **질문 처리 절차**:
    1. **질문의 핵심 내용 추출**:
        - 질문을 형태소 단위로 분석하여 조사를 무시하고 핵심 키워드만 추출합니다. 
        - 질문의 형태가 다르더라도 문맥의 의도가 같으면 동일한 질문으로 간주합니다.
        - 예를 들어, "개별소비세법 1조 알려줘" 와 "개별소비세법 1조는 뭐야" 와 "개별소비세법 1조의 내용은?"는 동일한 질문으로 간주합니다.
        - 예를 들어, "소득세는 무엇인가요?"와 "소득세가 무엇인가요?"는 동일한 질문으로 간주합니다.
    2. **관련 세법 조항 검색**:
        - 질문의 핵심 키워드와 가장 관련 있는 세법 조항이나 시행령을 우선적으로 찾습니다.
        - 필요한 경우, 질문과 연관된 추가 조항도 검토하여 답변의 완성도를 높입니다.
    3. **질문 유형 판단**:
        - **정의 질문**: 특정 용어나 제도의 정의를 묻는 경우.
        - **절차 질문**: 특정 제도의 적용이나 신고 방법을 묻는 경우.
        - **사례 질문**: 구체적인 상황에 대한 세법 해석을 요청하는 경우.
    4. **답변 생성**:
        - 법률 조항에관한 질문이라면 그 조항에 관한 전체 내용을 가져온다.
        - 예를들어 '개별소비세법 1조의 내용'이라는 질문을 받으면 개별소비세법 1조의 조항을 전부 다 답변한다.
        - 질문 유형에 따라 관련 정보를 구조적으로 작성하며, 중요 세법 조문과 요약된 내용을 포함합니다.
        - 비전문가도 이해할 수 있도록 용어를 친절히 설명합니다.

    **답변 작성 가이드라인**:
    - **간결성**: 답변은 간단하고 명확하게 작성하되, 법 조항에 관한 질문일 경우 관련 법 조문의 전문을 명시합니다.
    - **구조화된 정보 제공**:
        - 세법 조항 번호, 세법 조항의 정의, 시행령, 관련 규정을 구체적으로 명시합니다.
        - 복잡한 개념은 예시를 들어 설명하거나, 단계적으로 안내합니다.
    - **신뢰성 강조**:
        - 답변이 법적 조언이 아니라 정보 제공 목적임을 명확히 알립니다.
        - "이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다."를 추가합니다.
    - **정확성**:
        - 법령 및 법률에 관한질문은 추가적인 내용없이 한가지 content에 집중하여 답변한다.
        - 법조항에대한 질문은 시행령이나 시행규칙보단 해당법에서 가져오는것에 집중한다.
      
    **추가적인 사용자 지원**:
    - 답변 후 사용자에게 주제와 관련된 후속 질문 두 가지를 제안합니다.
    - 후속 질문은 사용자가 더 깊이 탐구할 수 있도록 설계하며, 각 질문 앞뒤에 한 줄씩 띄어쓰기를 합니다.

    **예외 상황 처리**:
    - 사용자가 질문을 모호하게 작성한 경우:
        - "질문이 명확하지 않습니다. 구체적으로 어떤 부분을 알고 싶으신지 말씀해 주시겠어요?"와 같은 문구로 추가 정보를 요청합니다.
    - 질문이 세법과 직접 관련이 없는 경우:
        - "이 질문은 제가 학습한 대한민국 세법 범위를 벗어납니다."라고 알리고, 세법과 관련된 새로운 질문을 유도합니다.

    **추가 지침**:
    - 개행문자 두 개 이상은 절대 사용하지 마세요.
    - 질문 및 답변에서 사용된 세법 조문은 최신 데이터에 기반해야 합니다.
    - 질문이 복합적인 경우, 각 하위 질문에 대해 별도로 답변하거나, 사용자에게 우선순위를 확인합니다.

    **예시 답변 템플릿**:
    - "질문에 대한 답변: ..."
    - "관련 세법 조항: ..."
    - "추가 설명: ..."

    {context}
    """),
    ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser

In [9]:
print(chain.invoke("법인세법 제1조 에 대해 알려줘"))
retriever.invoke("법인세법 제1조 에 대해 알려줘")

법인세법 제1조는 법인세법의 목적에 관한 조항입니다. 이 조항은 법인세법이 어떤 목적을 가지고 있으며, 그 법적 근거와 적용 범위를 명시하고 있습니다. 그러나 현재 제공된 데이터에는 법인세법 제1조의 전문이 포함되어 있지 않으므로, 법인세법 제1조의 구체적인 내용은 법령집이나 관련 법률 문서를 참조하시길 권장합니다.

다음은 법인세법 제1조의 일반적인 설명입니다:
- **목적**: 법인세법은 법인의 소득에 대한 과세를 규정하며, 이를 통해 국가 재정의 건전한 운영을 도모하고 있습니다.

이 답변은 법인세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.

추가로 궁금하신 부분이 있다면, 다음과 같은 후속 질문을 고려해 보세요:

1. 법인세법의 주요 조항과 그 적용 방법은 어떻게 되나요?

2. 법인세 신고 절차에 대해 설명해 주세요.


[Document(metadata={'description': '법인세법 시행규칙 에 관한 문서입니다.', 'keyword': '법인세법 시행규칙', 'source': '법인세법_시행규칙.pdf'}, page_content='법인세법 시행규칙\n \n기획재정부    (국제조세제도과   (외국법인)) 044-215-4652\n기획재정부    (법인세제과) 044-215-4221\n       제1장 총칙'),
 Document(metadata={'description': '법인세법 시행령 제1조에 관한 문서입니다.', 'keyword': '법인세법 시행령', 'source': '법인세법_시행령.pdf'}, page_content='제1조    (목적) 이 영은 「법인세법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.'),
 Document(metadata={'description': '조세범 처벌절차법 시행령 에 관한 문서입니다.', 'keyword': '조세범 처벌절차법 시행령', 'source': '조세범_처벌절차법_시행령.pdf'}, page_content='조세범 처벌절차법 시행령\n \n기획재정부    (조세법령운용팀) 044-215-4151\n       제1장 총칙')]

In [11]:
print(chain.invoke("연봉 4000에 회사를 다니는 근로자인데 개인사업도 하고있어. 개인사업으로는 올해 3000만원의 소득이 이루어졌는데 연말정산을 어떻게 해야할까?"))

연봉 4000만 원을 받는 직장인이면서 개인사업으로 3000만 원의 소득이 발생했다면, 연말정산과 종합소득세 신고를 모두 고려해야 합니다. 각각의 절차를 간단히 설명하겠습니다.

1. **연말정산**:
   - 연말정산은 주로 근로소득에 대해 회사에서 원천징수한 세금을 정산하는 과정입니다.
   - 회사가 제공한 근로소득 원천징수영수증을 바탕으로 소득 공제 및 세액 공제를 적용받아, 납부할 세액을 계산합니다.
   - 연말정산 결과에 따라 추가로 세금을 납부하거나 환급받을 수 있습니다.

2. **종합소득세 신고**:
   - 개인사업 소득은 종합소득세 신고 대상에 포함됩니다.
   - 사업소득 3000만 원에 대해 사업소득금액을 계산하고, 이를 포함해 종합소득 과세표준을 산정하게 됩니다.
   - 사업소득에 대한 소득 공제 및 세액 공제를 적용받아 최종적으로 납부할 세액을 계산합니다.
   - 종합소득세 신고는 다음 해 5월 31일까지 해야 하며, 이때 근로소득과 사업소득을 합산하여 신고합니다.

**추가 설명**:
- 종합소득세 신고 시에는 근로소득과 사업소득을 합산하여 종합소득에 대한 세금을 계산하게 됩니다.
- 사업소득에 적용할 수 있는 소득공제 항목을 잘 활용하여 세액을 줄일 수 있도록 준비하세요.
- 세금 신고 및 절세 전략에 대해 보다 정확한 조언이 필요하다면 세무사와 상담하는 것이 좋습니다.

이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.

**후속 질문 제안**:

연말정산에서 소득공제를 최대화하는 방법은 무엇일까요?

종합소득세 신고 시 유의할 점은 무엇인가요?


In [12]:
print(chain.invoke("현재 농사를 법인회사 형태로 소득을 벌어들이고있는데 세금혜택이있을까?"))

농사를 법인회사의 형태로 운영하는 경우, 조세특례제한법에 따라 특정한 세금 혜택을 받을 수 있습니다. 특히 조세특례제한법 제68조는 농업회사법인에 대한 법인세 면제 및 감면에 관한 규정을 두고 있습니다. 여기 몇 가지 주요 내용을 요약하여 설명드리겠습니다:

1. **법인세 면제**: 농업회사법인이 식량작물재배업에서 발생하는 소득 전액과 그 외의 작물재배업에서 발생하는 소득 중 대통령령으로 정하는 범위의 금액에 대하여 법인세를 면제받을 수 있습니다. 이 혜택은 2026년 12월 31일까지 적용됩니다.

2. **법인세 감면**: 작물재배업 외의 소득에 대해서는 최초 소득 발생 과세연도와 그 다음 4년 이내에 끝나는 과세연도까지 법인세의 50%를 감면받을 수 있습니다.

3. **양도소득세 감면**: 농업인이 농업회사법인에 현물출자하여 발생하는 소득에 대해서도 양도소득세의 100%를 감면받을 수 있습니다. 다만, 특정한 경우에 한하여 적용됩니다.

4. **배당소득에 대한 소득세 면제**: 농업회사법인에 출자한 거주자가 받는 배당소득 중 식량작물재배업소득에서 발생한 배당소득 전액에 대해서는 소득세를 면제받습니다.

5. **신청 필요**: 이러한 혜택을 받기 위해서는 대통령령으로 정하는 바에 따라 신청을 해야 합니다.

이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.

추가적으로 궁금한 점이 있으시면 다음과 같은 질문을 고려해 보실 수 있습니다:

- 농업회사법인의 법인세 감면 신청 절차는 어떻게 되나요?

- 농업회사법인의 현물출자 시 양도소득세 감면 조건은 무엇인가요?


## 평가

In [13]:
from ragas import EvaluationDataset, RunConfig, evaluate
from datasets import Dataset
from ragas.metrics import (LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

# LangChain 모델 래핑
langchain_model = LangchainLLMWrapper(model)

# 테스트 데이터 준비
test_data = [
    # {
    #     "question": "주세법의 목적은 무엇인가요?",
    #     "answer": chain.invoke("주세법의 목적은 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("주세법의 목적은 무엇인가요?")],
    #     "ground_truths": ["주세의 과세 요건 및 절차를 규정함으로써 주세를 공정하게 과세하고, 납세의무의 적정한 이행을확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다."],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("주세법의 목적은 무엇인가요?")])
    # },
    # {
    #     "question": "인적공제에서 공제금액을 알려주세요.",
    #     "answer": chain.invoke("인적공제에서 공제금액을 알려주세요."),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("인적공제에서 공제금액을 알려주세요.")],
    #     "ground_truths": ["기본공제대상자 1명당 150만원. 기본공제대상자가 다음에 해당되는 경우 기본공제 외에 추가로 공제, 경로우대자(70세 이상)인 경우 1명당 연 100만원, 장애인인 경우 1명당 연 200만원, 종합소득금액이 3천만원 이하인 거주자가 배우자가 없는 여성으로서 기본공제대상부양가족이 있는세대주이거나, 배우자가 있는 여성근로자인 경우 연 50만원, 배우자가 없는 근로자가 기본공제대상 직계비속 또는 입양자가 있는 경우 연 100만원(부녀자공제와 중복 배제：한부모 공제를 우선 적용)"],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("인적공제에서 공제금액을 알려주세요.")])
    # },
    # {
    #     "question": "교육세법법 제1조가 무엇인가요?",
    #     "answer": chain.invoke("교육세법법 제1조가 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("교육세법법 제1조가 무엇인가요?")],
    #     "ground_truths": ["목적은 교육의 질적 향상을 도모하기 위하여 필요한 교육재정의 확충에 드는 재원을 확보함을 목적이다."],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("교육세법법 제1조가 무엇인가요?")])
    # },
    # {
    #     "question": "개정세법 중 기업세금 감면제도의 개정 내용 요약해서 알려줘.",
    #     "answer": chain.invoke("개정세법 중 기업세금 감면제도의 개정 주요요내용 요약해서 알려줘."),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("개정세법 중 기업세금 감면제도의 개정 내용 요약해서 알려줘.")],
    #     "ground_truths": ["창업중소기업 세액감면 제도 합리화, 연구･인력개발비에 대한 세액공제 확대, R&D 출연금에 대한 과세특례 인정범위 확대, 기술혁신형 중소기업 주식취득에 대한 세액공제 합리화, 성과공유 중소기업 경영성과급에 대한 세액공제 등의 적용기한 연장 및 재설계, 통합투자세액공제 제도 합리화, 지방이전지원세제 제도 정비, 소기업･소상공인 공제부금에 대한 소득공제 한도 확대, 성실사업자 등에 대한 의료비 등 세액공제 사후관리 합리화"],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("개정세법 중 기업세금 감면제도의 개정 내용 요약해서 알려줘.")])
    # },
    {
        "question": "부담부 증여로 주택 취득시 장기주택저당차입금 이자상환액 공제가 가능한가가?",
        "answer": chain.invoke("부담부 증여로 주택 취득시 장기주택저당차입금 이자상환액 공제가 가능한가가?"),
        "contexts": [doc.page_content for doc in retriever.get_relevant_documents("부담부 증여로 주택 취득시 장기주택저당차입금 이자상환액 공제가 가능한가가?")],
        "ground_truths": ["증여등기일로부터 3개월 이내에 해당 주택에 저당권을 설정하고 상환기간이 15년 이상인 장기주택저당차입금을 대출받아 증여재산에 담보된 채무를 상환하는 경우 해당 채무액의 범위 내에서 이자상환액 소득공제가 가능합니다."],
        "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("부담부 증여로 주택 취득시 장기주택저당차입금 이자상환액 공제가 가능한가가?")])
    }

]

# Dataset 생성
dataset = Dataset.from_list(test_data)

# 평가 실행
result = evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
    ],
    llm=langchain_model,
)

# 결과 출력
print(result)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_29980\1933053170.py:45: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  "contexts": [doc.page_content for doc in retriever.get_relevant_documents("부담부 증여로 주택 취득시 장기주택저당차입금 이자상환액 공제가 가능한가가?")],


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

{'faithfulness': 0.7778, 'answer_relevancy': 0.9919, 'context_precision': 1.0000, 'context_recall': 0.5556}


In [14]:
# GPT-4o 모델을 사용하여 평가 
model_name = "gpt-4o"
model = ChatOpenAI(model=model_name)
langchain_model = LangchainLLMWrapper(model)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)
metrics = [
    LLMContextRecall(llm=langchain_model),
    LLMContextPrecisionWithReference(llm=langchain_model),
    Faithfulness(llm=langchain_model),
    AnswerRelevancy(llm=langchain_model, embeddings=eval_embedding)
]
result = evaluate(dataset=dataset, metrics=metrics)
result

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

{'context_recall': 0.5556, 'llm_context_precision_with_reference': 1.0000, 'faithfulness': 0.6667, 'answer_relevancy': 0.9395}

In [15]:
# DataFrame 생성
df_result = pd.DataFrame(test_data)
# 최대 열 너비 설정
pd.set_option('display.max_colwidth', None)
# 최대 행 수 설정
pd.set_option('display.max_rows', None)

df_result

,question,answer,contexts,ground_truths,reference
0,부담부 증여로 주택 취득시 장기주택저당차입금 이자상환액 공제가 가능한가가?,"질문에 대한 답변: 부담부 증여로 주택을 취득한 경우 장기주택저당차입금 이자상환액 공제가 가능합니다.\n\n관련 세법 조항: 증여등기일로부터 3개월 이내에 해당 주택에 저당권을 설정하고, 상환기간이 15년 이상인 장기주택저당차입금을 대출받아 증여재산에 담보된 채무를 상환하는 경우 해당 채무액의 범위 내에서 이자상환액에 대한 소득공제를 받을 수 있습니다.\n\n추가 설명: 이는 원천세과-538, 2011.8.30.의 해석에 따른 것으로, 증여로 인한 부담부 채무를 명확히 하고 적절한 저당권 설정과 상환기간 등의 요건을 충족해야 합니다.\n\n이 답변은 세법 관련 정보를 바탕으로 작성되었으며, 구체적인 상황에 따라 전문가의 추가 조언이 필요할 수 있습니다.\n\n추가적으로 관심을 가질 수 있는 주제:\n \n - 장기주택저당차입금 이자상환액 공제를 받을 수 있는 다른 조건은 무엇인가요?\n \n - 부담부 증여 외에 다른 방식으로 주택을 취득할 때의 세금 혜택은 무엇인가요?","[•• 49 Q15 부담부 증여로 주택 취득시 장기주택저당차입금 이자상환액 공제가 가능한지? ⚫증여등기일로부터 3개월 이내에 해당 주택에 저당권을 설정하고 상환기간이 15년 이상인 장기주택저당차입금을 대출받아 증여재산에 담보된 채무를 상환하는 경우 해당 채무액의 범위 내에서 이자상환액 소득공제가 가능합니다. (원천세과-538, 2011.8.30.) Q16 오피스텔을 구입하여 주거용으로 사용하는 경우 오피스텔 구입을 위해 금융기관으로부터 차입한 저당 차입금에 대해 이자상환액 공제를 적용받을 수 있는지? ⚫안됩니다. 오피스텔은 건축법상 업무시설로 장기주택저당차입금 이자상환액 공제 대상 주택에 해당하지 않습니다. (서면인터넷방문상담1팀-567, 2004.4.19.) Q17 장기주택저당차입금 이자상환액 공제 요건을 갖추었으나 해당 주택에 실제로 거주하지 않는 경우에도 소득공제 가능한지? ⚫주택의 소유주가 세대주인 경우에는 해당 주택에 거주하지 않아도 장기주택 저당차입금 이자상환액 공제를 받을 수 있으나, 세대주가 아닌 세대원인 근로자인 경우에는 해당 주택에 실제 거주하는 경우에만 소득공제를 받을 수 있습니다. (소득세법 제52조 제⑤항 제3호), •• Q18 1주택을 보유하고 있는 사람이 아파트 분양권을 담보로 대출을 받은 경우 장기주택저당차입금 이자상환액 공제가 가능한지? ⚫안됩니다. 주택분양권 등에 대한 장기주택저당차입금 이자상환액 공제는 무주택자만 공제 대상입니다. (소득세법 제52조 제⑤항 제4호) Q19 주택분양권을 취득하고 중도금 대출을 받을 때 추후 주택 완공시 장기주택 저당차입금으로 전환한 것을 조건으로 차입하면 장기주택저당차입금으로 보는지? ⚫예, 무주택 세대주가 6억원 이하인 주택분양권 또는 조합입주권을 취득하고, 주택 완공시 장기주택저당차입금으로 전환할 것을 조건으로 금융회사 등 또는 주택도시기금법에 따른 주택도시기금으로부터 차입한 경우 그 차입일(차입조건을 새로 변경한 경우에는 그 변경일)부터 그 주택의 소유권 보존등기일까지 그 차입금을 장기주택저당차입금으로 보는 것입니다. 다만, 주택분양권을 둘 이상 보유하게 된 경우에는 그 보유기간이 속하는 과세기간에는 이를 적용하지 않습니다. (소득세법 제52조 제⑤항 제4호) Q20 기준시가가 공시되기 전에 차입금을 차입한 경우 취득한 주택의 기준시가는 어떻게 판단하는지?, •• 29 05 주택분양권에 대한 장기주택저당금 이자상환액 소득공제 ❚사실관계 ⚫’23.2월 ｢소득세법｣ 제52조 제5항 제4호에 해당하는 장기주택저당차입금에 대한 이자를 시행사가 대위변제하고 주택 완공시 이를 지급하기로 하는 아파트분양권을 취득하였으나, - 시행사가 자금부족으로 이자후불제 혜택이 중단되어 ’23.3월부터 3개월간 직접 은행에 이자를 납부하였고, 주택이 완공되어 ’23.6월부터는 특별보금자리론을 이용하여 장기주택저당차입금으로 전환함 ❚관련법령 ✦근로소득이 있는 거주자로서 주택을 소유하지 아니하거나 1주택을 보유한 세대의 세대주(세대주가 이 항, 제4항 및 「조세특례제한법」 제87조제2항에 따른 공제를 받지 아니하는 경우에는 세대의 구성원 중 근로소득이 있는 자를 말하며, 대통령령으로 정하는 외국인을 포함한다)가 취득 당시 제99조 제1항에 따른 주택의 기준시가가 6억원 이하인 주택을 취득하기 위하여 ~ (소법 §52⑤) 4. 무주택자인 세대주가 「주택법」에 따른 사업계획의 승인을 받아 건설되는 주택 (「주택법」에 따른 주택조합 및 「도시 및 주거환경정비법」에 따른 정비사업조합의 조합원이 취득하는 주택 또는 그 조합을 통하여 취득하는 주택을 포함한다. 이하 이 호에서 같다)을 취득할 수 있는 권리(이하 이 호에서 “주택분양권”이라 한다)로서 대통령령으로 정하는 가격이 6억원 이하인 권리를 취득하고 ~ 중략 ~그 주택의 소유권보존등기일까지 그 차입금을 장기주택저당차입금으로 본다. 다만, 거주자가 주택분양권을 둘 이상 보유하게 된 경우에는 그 보유기간이 속하는 과세기간에는 적용하지 아니한다. ❚해석사례 (사전-2024-법규소득-0032, ’24.6.26.) ｢소득세법｣ 제52조 제5항 제4호에 따른 장기주택저당차입금에 대한 이자를 무주택자인 세대주가 지급한 경우에는 해당 과세기간에 지급한 이자상환액을 같은 법 같은 조에 따른 한도액의 범위 내에서 해당 과세기간의 근로소득금액에서 공제할 수 있는 것임]",[증여등기일로부터 3개월 이내에 해당 주택에 저당권을 설정하고 상환기간이 15년 이상인 장기주택저당차입금을 대출받아 증여재산에 담보된 채무를 상환하는 경우 해당 채무액의 범위 내에서 이자상환액 소득공제가 가능합니다.],"•• 49 Q15 부담부 증여로 주택 취득시 장기주택저당차입금 이자상환액 공제가 가능한지? ⚫증여등기일로부터 3개월 이내에 해당 주택에 저당권을 설정하고 상환기간이 15년 이상인 장기주택저당차입금을 대출받아 증여재산에 담보된 채무를 상환하는 경우 해당 채무액의 범위 내에서 이자상환액 소득공제가 가능합니다. (원천세과-538, 2011.8.30.) Q16 오피스텔을 구입하여 주거용으로 사용하는 경우 오피스텔 구입을 위해 금융기관으로부터 차입한 저당 차입금에 대해 이자상환액 공제를 적용받을 수 있는지? ⚫안됩니다. 오피스텔은 건축법상 업무시설로 장기주택저당차입금 이자상환액 공제 대상 주택에 해당하지 않습니다. (서면인터넷방문상담1팀-567, 2004.4.19.) Q17 장기주택저당차입금 이자상환액 공제 요건을 갖추었으나 해당 주택에 실제로 거주하지 않는 경우에도 소득공제 가능한지? ⚫주택의 소유주가 세대주인 경우에는 해당 주택에 거주하지 않아도 장기주택 저당차입금 이자상환액 공제를 받을 수 있으나, 세대주가 아닌 세대원인 근로자인 경우에는 해당 주택에 실제 거주하는 경우에만 소득공제를 받을 수 있습니다. (소득세법 제52조 제⑤항 제3호)\n•• ⚫예, 근로소득이 있는 거주자로서 주택을 소유하지 아니하거나 1주택을 보유한 세대의 세대주가 기준시가 5억원 이하의 주택을 취득하기 전에 그 주택에 저당권을 설정하여 금융기관 등으로부터 만기 15년 이상의 주택자금을 차입한 후 즉시 본인에게로 소유권을 이전등기(공동소유 포함)하는 경우에는 공제가 가능합니다. (법령해석소득-217, 2020.4.6.) Q41 주택담보대출로 차입한 장기주택저당차입금의 대출약정사항은 법령에서 규정한 비거치식 분할상환방식에 해당하지 않으나, 대출약정사항보다 많은 금액을 상환하여 요건을 충족하게 되는 경우 비거치식 분할상환방식에 해당하는 것으로 보아 법령에 규정된 한도금액을 적용받을 수 있는지? ⚫예, 법령에서 규정한 “비거치식 분할상환방식”이란 차입일이 속하는 과세기간의 다음 과세기간부터 차입금 상환기간의 말일이 속하는 과세기간까지 매년 “차입금의 100분의70/상환기간 연수”이상의 차입금을 상환하는 경우를 의미 하는 것입니다. (서면-2024-법규소득-0456, 2024.6.27.) Q42 장기주택저당차입금 이자상환액의 소득공제 대상 주택이 재건축으로 멸실되어 조합원입주